In [1]:

import gzip
import os
import pickle
from urllib.request import urlretrieve

import nengo
import nengo_dl
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
try:
    import requests
    has_requests = True
except ImportError:
    has_requests = False

import nengo_loihi
import keras

W1209 21:00:28.002981 13712 module_wrapper.py:139] From c:\winpython\python-3.6.5.amd64\lib\site-packages\nengo_dl\compat.py:26: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1209 21:00:28.009979 13712 module_wrapper.py:139] From c:\winpython\python-3.6.5.amd64\lib\site-packages\nengo_dl\__init__.py:38: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W1209 21:00:28.010978 13712 module_wrapper.py:139] From c:\winpython\python-3.6.5.amd64\lib\site-packages\nengo_dl\__init__.py:38: The name tf.logging.WARN is deprecated. Please use tf.compat.v1.logging.WARN instead.

W1209 21:00:28.011978 13712 module_wrapper.py:139] From c:\winpython\python-3.6.5.amd64\lib\site-packages\nengo_dl\__init__.py:43: The name tf.disable_v2_behavior is deprecated. Please use tf.compat.v1.disable_v2_behavior instead.

W1209 21:00:28.012977 13712 deprecation.py:323] From c:\winpython\python-3.6.5.amd64\lib\site-packages\t

In [2]:
gpu_options = tf.GPUOptions(allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


In [3]:
(train_images, train_labels), (test_images, test_labels) = (
    keras.datasets.cifar10.load_data())


# flatten images
train_images = train_images.reshape((train_images.shape[0], -1))
test_images = test_images.reshape((test_images.shape[0], -1))




train_data = [train_images/254, train_labels]
test_data = [test_images/254, test_labels]

    
# plot some examples
#for i in range(3):
#    plt.figure()
#    plt.imshow(np.reshape(train_data[0][i], (3, 32, 32)))
#    plt.axis('off')
#    plt.title(str(np.argmax(train_data[1][i])));

In [4]:
train_data[1] = [x[0] for x in train_data[1]]
test_data[1] = [x[0] for x in test_data[1]]

for data in (train_data, test_data):
    one_hot = np.zeros((data[0].shape[0], 10))
    one_hot[np.arange(data[0].shape[0]), data[1]] = 1
    data[1] = one_hot



In [5]:
def conv_layer(x, *args, activation=True, **kwargs):
    # create a Conv2D transform with the given arguments
    conv = nengo.Convolution(*args, channels_last=True, **kwargs)

    if activation:
        # add an ensemble to implement the activation function
        layer = nengo.Ensemble(conv.output_shape.size, 1).neurons
    else:
        # no nonlinearity, so we just use a node
        layer = nengo.Node(size_in=conv.output_shape.size)

    # connect up the input object to the new layer
    nengo.Connection(x, layer, transform=conv)

    # print out the shape information for our new layer
    print("LAYER")
    print(conv.input_shape.shape, "->", conv.output_shape.shape)

    return layer, conv

In [6]:
dt = 0.001  # simulation timestep
presentation_time = 0.1  # input presentation time
max_rate = 100  # neuron firing rates
# neuron spike amplitude (scaled so that the overall output is ~1)
amp = 1 / max_rate
# input image shape
input_shape = (32, 32, 3)
n_parallel = 1

with nengo.Network(seed=0) as net:
    # set up the default parameters for ensembles/connections
    nengo_loihi.add_params(net)
    net.config[nengo.Ensemble].neuron_type = (
        nengo.LIF(amplitude=amp))
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([max_rate])
    net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
    net.config[nengo.Connection].synapse = None

    # the input node that will be used to feed in input images
    inp = nengo.Node(
        nengo.processes.PresentInput(test_data[0], presentation_time),
        size_out= 32 * 32 * 3)

    # the output node provides the 10-dimensional classification
    out = nengo.Node(size_in=10)
    for _ in range(n_parallel):
        # build parallel copies of the network
        layer, conv = conv_layer(
            inp, 1, input_shape, kernel_size=(1, 1),
            init=np.ones((1, 1, 3, 1)))
        # first layer is off-chip to translate the images into spikes
        net.config[layer.ensemble].on_chip = False
        layer, conv = conv_layer(layer, 8, conv.output_shape,
                                 strides=(2, 2))
        #layer, conv = conv_layer(layer, 24, conv.output_shape,
        #                         strides=(2, 2))
        #layer, conv = conv_layer(layer, 48, conv.output_shape,
        #                         strides=(2, 2))
        nengo.Connection(layer, out, transform=nengo_dl.dists.Glorot())

    out_p = nengo.Probe(out)
    out_p_filt = nengo.Probe(out, synapse=nengo.Alpha(0.01))

LAYER
(32, 32, 3) -> (32, 32, 1)
LAYER
(32, 32, 1) -> (15, 15, 8)


In [7]:
# set up training data
minibatch_size = 200
train_data = {inp: train_data[0][:, None, :],
              out_p: train_data[1][:, None, :]}

# for the test data evaluation we'll be running the network over time
# using spiking neurons, so we need to repeat the input/target data
# for a number of timesteps (based on the presentation_time)
test_data = {
    inp: np.tile(test_data[0][:1000, None, :],
                 (1, int(presentation_time / dt), 1)),
    out_p_filt: np.tile(test_data[1][:1000, None, :],
                        (1, int(presentation_time / dt), 1))
}

In [8]:
def crossentropy(outputs, targets):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=outputs, labels=targets))


def classification_error(outputs, targets):
    return 100 * tf.reduce_mean(
        tf.cast(tf.not_equal(tf.argmax(outputs[:, -1], axis=-1),
                             tf.argmax(targets[:, -1], axis=-1)),
                tf.float32))


In [9]:
do_training = True

with nengo_dl.Simulator(net, minibatch_size=minibatch_size, seed=0) as sim:
    if do_training:
        #print("error before training: %.2f%%" %
        #      sim.loss(test_data, {out_p_filt: classification_error}))

        # run training
        sim.train(train_data, tf.train.RMSPropOptimizer(learning_rate=0.001),
                  objective={out_p: crossentropy}, n_epochs=5)

        print("error after training: %.2f%%" %
              sim.loss(test_data, {out_p_filt: classification_error}))

        sim.save_params("./mnist_params4")
    else:
        #download("mnist_params.data-00000-of-00001",
        #         "1BaNU7Er_Q3SJt4i4Eqbv1Ln_TkmmCXvy")
        #download("mnist_params.index", "1w8GNylkamI-3yHfSe_L1-dBtvaQYjNlC")
        #download("mnist_params.meta", "1JiaoxIqmRupT4reQ5BrstuILQeHNffrX")
        sim.load_params("./mnist_params4")

    # store trained parameters back into the network
    sim.freeze_params(net)


Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Training finished in 0:01:03 (loss: 1.4845)                                    
Calculation finished in 0:00:14                                                
error after training: 89.00%


W1209 21:00:38.023187 13712 module_wrapper.py:139] From c:\winpython\python-3.6.5.amd64\lib\site-packages\nengo_dl\simulator.py:160: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W1209 21:00:38.037181 13712 module_wrapper.py:139] From c:\winpython\python-3.6.5.amd64\lib\site-packages\nengo_dl\tensor_graph.py:200: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1209 21:00:38.042180 13712 module_wrapper.py:139] From c:\winpython\python-3.6.5.amd64\lib\site-packages\nengo_dl\tensor_graph.py:205: The name tf.train.get_or_create_global_step is deprecated. Please use tf.compat.v1.train.get_or_create_global_step instead.

W1209 21:00:38.055173 13712 module_wrapper.py:139] From c:\winpython\python-3.6.5.amd64\lib\site-packages\nengo_dl\tensor_graph.py:229: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1209 21:00:38.056173 13712 module_wrapper.py:139] From c:\winpyt

In [25]:
for conn in net.all_connections:
    conn.synapse = 0.016


with nengo_dl.Simulator(net, minibatch_size=minibatch_size) as sim:
    print("error w/ synapse: %.2f%%" %
          sim.loss(test_data, {out_p_filt: classification_error}))

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Calculation finished in 0:00:11                                                
error w/ synapse: 52.80%


In [ ]:
n_presentations = 50
with nengo_loihi.Simulator(net, dt=dt, precompute=False) as sim:
    # if running on Loihi, increase the max input spikes per step
    if 'loihi' in sim.sims:
        sim.sims['loihi'].snip_max_spikes_per_step = 120

    # run the simulation on Loihi
    sim.run(n_presentations * presentation_time)

    # check classification error
    step = int(presentation_time / dt)
    output = sim.data[out_p_filt][step - 1::step]
    correct = 100 * (np.mean(
        np.argmax(output, axis=-1)
        != np.argmax(test_data[out_p_filt][:n_presentations, -1],
                     axis=-1)
    ))
    print("loihi error: %.2f%%" % correct)

In [ ]:
n_presentations = 50
with nengo.Simulator(net, dt=dt) as sim:
    # if running on Loihi, increase the max input spikes per step
    #if 'loihi' in sim.sims:
    #    sim.sims['loihi'].snip_max_spikes_per_step = 120

    # run the simulation on Loihi
    sim.run(n_presentations * presentation_time)

    # check classification error
    step = int(presentation_time / dt)
    output = sim.data[out_p_filt][step - 1::step]
    correct = 100 * (np.mean(
        np.argmax(output, axis=-1)
        != np.argmax(test_data[out_p_filt][:n_presentations, -1],
                     axis=-1)
    ))
    print("loihi error: %.2f%%" % correct)

In [ ]:
n_plots = 10
plt.figure()

plt.subplot(2, 1, 1)
images = test_data[inp].reshape(-1, 28, 28, 1)[::step]
ni, nj, nc = images[0].shape
allimage = np.zeros((ni, nj * n_plots, nc), dtype=images.dtype)
for i, image in enumerate(images[:n_plots]):
    allimage[:, i * nj:(i + 1) * nj] = image
if allimage.shape[-1] == 1:
    allimage = allimage[:, :, 0]
plt.imshow(allimage, aspect='auto', interpolation='none', cmap='gray')

plt.subplot(2, 1, 2)
plt.plot(sim.trange()[:n_plots * step], sim.data[out_p_filt][:n_plots * step])
plt.legend(['%d' % i for i in range(10)], loc='best');